In [28]:
import numpy as np
import pandas as pd

In [29]:
dataset = pd.read_csv("employee_data.csv")
print(dataset.head())

   Unnamed: 0  id groups  age  healthy_eating  active_lifestyle  salary
0           0   0      A   36               5                 5    2297
1           1   1      A   55               3                 5    1134
2           2   2      A   61               8                 1    4969
3           3   3      O   29               3                 6     902
4           4   4      O   34               6                 2    3574


In [30]:
dataset = dataset.drop("Unnamed: 0", 1)
dataset = dataset.drop("id", 1)
print(dataset.head())

  groups  age  healthy_eating  active_lifestyle  salary
0      A   36               5                 5    2297
1      A   55               3                 5    1134
2      A   61               8                 1    4969
3      O   29               3                 6     902
4      O   34               6                 2    3574


In [31]:
dataset = dataset.as_matrix()
print(dataset)

[['A' 36 5 5 2297]
 ['A' 55 3 5 1134]
 ['A' 61 8 1 4969]
 ...
 ['O' 49 9 7 4158]
 ['AB' 56 6 7 2414]
 ['B' 64 4 9 788]]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [32]:
x = dataset[:, :-1]
y = dataset[:, -1]

print(x[0], y[0])

['A' 36 5 5] 2297


In [33]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown="ignore", dtype=np.int8)
enc.fit(x[:, [0]])
x_cat = enc.transform(x[:, [0]]).toarray()
print(x_cat.shape)
print(x_cat[0])

(1000, 4)
[1 0 0 0]


In [34]:
x_cat = np.append(x_cat, x[:, 1:], axis=1)
print(x_cat.shape)
print(x_cat[0])

(1000, 7)
[1 0 0 0 36 5 5]


In [35]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_cat, y, test_size=0.3, random_state=42)

In [36]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(x_train, y_train)
score = regr.score(x_test, y_test)
print("Score: ", score)

Score:  0.9034071894250446


In [37]:
import pickle

pickle.dump(regr, open("model.pkl", "wb"))
regr = pickle.load(open("model.pkl", "rb"))

sample = np.array([1, 0, 0, 0, 40, 10, 10])
print(regr.predict([sample]))

[3759.76073213]
